In [1]:
from ast import literal_eval
REACH_DIR = './data/uid_reach/'
uid_reach = sc.textFile(REACH_DIR).map(literal_eval).cache()
uid_reach.count()

33304

In [2]:
# load safe key/token list
import requests
import json
from hashlib import md5
trackers = set(json.loads(requests.get('https://cdn.cliqz.com/anti-tracking/whitelist/tracker_domains.json').text).keys())

def isTracker(domain):
    return md5(domain).hexdigest()[:16] in trackers

safekeys = json.loads(requests.get('https://cdn.cliqz.com/anti-tracking/whitelist/domain_safe_key.json').text)

def isSafeKey(domain, key):
    domain_hash = md5(domain).hexdigest()[:16]
    key_hash = md5(key).hexdigest()
    return domain_hash in safekeys and key_hash in safekeys[domain_hash].keys()

safetokens = json.loads(requests.get('https://cdn.cliqz.com/anti-tracking/whitelist/whitelist_tokens.json').text)

def isSafeToken(token):
    return md5(token).hexdigest() in safetokens


In [3]:
not_uids = set(['.doubleclick.net', 'CheckForPermission', 'HttpOnly', 'httponly', 'secure'])

def is_manual_safe(domain, key, value):
    return value.strip() in not_uids

## Make requests accessable to SparkSQL

In [55]:
from pyspark.sql import Row

requests = sc.textFile('./requests/').map(json.loads)

kv_types = ['req_cookies', 'req_headers', 'res_headers', 'urlencoded_form']
sub_tables = kv_types + ['found_urls']
request_cols = ['rid', 'method', 'scheme', 'host', 'path', 'port', 'res_status', 'text', 'tld', 'url']

def split_sub_tables(request):
    rid = request['rid']

    for url in request['found_urls']:
        yield 'found_urls', Row(rid=rid, source=url[0], url=url[1])
    for kv_type in kv_types:
        for kv in request[kv_type]:
            yield kv_type, Row(rid=rid, key=kv[0], value=kv[1])
    
    request_flat = {k: request.get(k, '') for k in request_cols}
    yield 'request', Row(**request_flat)

table_data = requests.flatMap(split_sub_tables).cache()
tables = {k: sqlContext.createDataFrame(table_data.filter(lambda r: r[0] == k).values())
          for k in sub_tables}
requests_table = sqlContext.createDataFrame(table_data.filter(lambda r: r[0] == 'request').values()).cache()

for k, v in tables.items():
    v.registerTempTable(k)
requests_table.registerTempTable('requests')

In [69]:
sqlContext.sql("""\
SELECT r.rid, r.url, f.source, f.url, c.value \
FROM requests AS r \
JOIN found_urls AS f ON r.rid = f.rid \
JOIN res_headers AS c ON r.rid = c.rid AND c.key = 'Set-Cookie' AND c.value LIKE '%utastes%'
WHERE r.tld = 'outbrain.com' \
""").toPandas()

,rid,url,source,url,value
0,45580,http://odb.outbrain.com/utils/get?url=http%3A%...,referer,http://www.bild.de/,[obuid=d52b2aa0-b7bb-43f6-92db-2462c90c00da;Pa...
1,45874,http://odb.outbrain.com/utils/get?url=http%3A%...,referer,http://www.stylebook.de/stars/curvy-model-ashl...,[obuid=1624191f-21c8-4bb1-a9dc-23d6fbb46667;Pa...
2,46492,http://odb.outbrain.com/utils/get?url=http%3A%...,referer,http://www.focus.de/digital/videos/schauen-sie...,[obuid=3a6bdfc3-69f8-4a96-8597-fe232ce20915;Pa...
3,7200,http://odb.outbrain.com/utils/get?url=http%3A%...,referer,http://www.vanityfair.com/news/2016/12/uber-an...,[obuid=1a7d3708-237f-47dc-ab62-457bd7c73b06;Pa...
4,67243,https://odb.outbrain.com/utils/get?url=http%3A...,referer,https://www.theguardian.com/uk-news/2017/jan/0...,[obuid=25a2a703-8102-4fd0-a892-e3c04b951a75;Pa...
5,66639,https://odb.outbrain.com/utils/get?url=http%3A...,referer,https://www.theguardian.com/commentisfree/2017...,[obuid=25a2a703-8102-4fd0-a892-e3c04b951a75;Pa...
6,67042,https://odb.outbrain.com/utils/get?url=http%3A...,referer,https://www.theguardian.com/cities/2017/jan/09...,[obuid=25a2a703-8102-4fd0-a892-e3c04b951a75;Pa...
7,66624,https://odb.outbrain.com/utils/get?url=http%3A...,referer,https://www.theguardian.com/commentisfree/2017...,[obuid=25a2a703-8102-4fd0-a892-e3c04b951a75;Pa...
8,46551,http://odb.outbrain.com/utils/get?url=http%3A%...,referer,http://www.focus.de/digital/videos/schauen-sie...,[obuid=3a6bdfc3-69f8-4a96-8597-fe232ce20915;Pa...
9,45886,http://odb.outbrain.com/utils/get?url=http%3A%...,referer,http://www.stylebook.de/stars/curvy-model-ashl...,[obuid=1624191f-21c8-4bb1-a9dc-23d6fbb46667;Pa...


## Make UID Tables

In [27]:
from urllib.parse import urlparse

def uid_dict(tup):
    (domain, uid), meta = tup
    meta['domain'] = domain
    meta['uid'] = uid
    # extra processing
    meta['non_fp_uniques'] = [elem for elem in meta['uniques_seen'] if not domain in elem[1]]
    meta['tp_domains'] = set([urlparse(elem[1]).netloc for elem in meta['non_fp_uniques']])
    return meta

def add_id(tup):
    d, id = tup
    d['id'] = id
    return d

def get_uid_class(host, source, key, val):
    if len(val) <= 4:
        return 'short'
    elif isSafeKey(host.encode('utf-8'), key.encode('utf-8')):
        return 'safekey'
    elif isSafeToken(val.encode('utf-8')):
        return 'safetoken'
    elif host in val or is_manual_safe(host, key, val):
        return 'manual'
    else:
        return 'uid'

def split_uid_tables(uid_data):
    uid_id = uid_data['id']
    uid_domain = uid_data['domain']
    for col in ['non_fp_uniques', 'uniques_seen']:
        for elem in uid_data[col]:
            yield col, Row(uid_id=uid_id, source=elem[0], url=elem[1])
    
    for col in ['tp_domains', 'unique_domains']:
        for domain in uid_data[col]:
            yield col, Row(uid_id=uid_id, domain=domain)
            
    for uid_part in uid_data['uid']:
        source, key, value = uid_part
        yield 'uid_parts', Row(uid_id=uid_id, domain=uid_domain, source=source, key=key, value=value,
                        classification=get_uid_class(uid_domain, source, key, value))
    
    yield 'uid', Row(uid_id=uid_id, domain=uid_domain, uid=str(uid_data['uid']),
                    **{k: len(uid_data[k]) for k in ['non_fp_uniques', 'uniques_seen', 'tp_domains', 'unique_domains']})
    
uid_table_data = uid_reach.map(uid_dict).zipWithIndex().map(add_id).flatMap(split_uid_tables).cache()
    
uid_tables = {k: sqlContext.createDataFrame(uid_table_data.filter(lambda r: r[0] == k).values())
          for k in ['uid', 'non_fp_uniques', 'uniques_seen', 'tp_domains', 'unique_domains', 'uid_parts']}

for k, v in uid_tables.items():
    v.registerTempTable(k)

In [48]:
sqlContext.sql("""SELECT \
u.uid_id, first(u.domain), first(u.non_fp_uniques) as non_fp_uniques, \
first(u.tp_domains) AS tp_domains, \
COUNT(p.key) AS uids,
COUNT(CASE WHEN p.source = 'cookie' THEN 1 ELSE NULL END) AS cookies, \
COUNT(CASE WHEN p.source = 'qs' THEN 1 ELSE NULL END) AS qs, \
COUNT(CASE WHEN p.source = 'ps' THEN 1 ELSE NULL END) AS ps \
FROM uid AS u \
LEFT JOIN uid_parts AS p ON u.uid_id = p.uid_id AND p.classification = 'uid' \
GROUP BY u.uid_id \
ORDER BY tp_domains DESC""").where('uids > 0').limit(10).toPandas()

,uid_id,"first(domain, false)",non_fp_uniques,tp_domains,uids,cookies,qs,ps
0,98,github.com,13,10,8,6,0,0
1,71,github.com,13,10,3,3,0,0
2,93,github.com,13,10,4,4,0,0
3,205,google-analytics.com,14,7,1,0,1,0
4,218,googlesyndication.com,12,7,1,0,1,0
5,136,youtube.com,11,7,2,0,0,0
6,137,googlesyndication.com,10,7,1,0,1,0
7,174,googlesyndication.com,17,7,1,0,1,0
8,148,youtube.com,11,7,1,0,0,0
9,186,googlesyndication.com,9,6,2,0,2,0


In [51]:
from IPython.display import display
check_id = 218
display(sqlContext.sql("""SELECT * FROM uid_parts WHERE uid_id = {}""".format(check_id)).toPandas())
sqlContext.sql("""SELECT * FROM non_fp_uniques WHERE uid_id = {}""".format(check_id)).toPandas()

,classification,domain,key,source,uid_id,value
0,short,googlesyndication.com,bas,qs,218,0
1,uid,googlesyndication.com,bos,qs,218,"1314,1440"
2,short,googlesyndication.com,lte,qs,218,1
3,short,googlesyndication.com,mc,qs,218,1
4,safetoken,googlesyndication.com,ss,qs,218,"2560,1440"


,source,uid_id,url
0,referer,218,https://ad.doubleclick.net/ddm/adi/N1203.amazo...
1,referer_url,218,https://www.youtube.com/ad_frame?id=watch-chan...
2,referer,218,https://googleads.g.doubleclick.net/pagead/ads...
3,referer_url,218,www.youtube.com/video/9bZkp7q19f0
4,referer_url,218,www.amazon.de/$0
5,referer_url,218,https://www.youtube.com/watch?v=9bZkp7q19f0
6,referer_url,218,www.youtube.com/watch?v=9bZkp7q19f0
7,referer_url,218,https://www.amazon.de/$0
8,referer,218,http://www.nytimes.com/2013/10/18/business/for...
9,referer_url,218,http://www.youtube.com/video/9bZkp7q19f0
